In [28]:
import sqlite3
import os
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from email.header import Header
from common.web_util import make_grid_tr
from common.log_util import *

In [32]:
## 데이터 읽어오기
def read_all_fares(db,sitecodes,scrap_date=None):
    if scrap_date is None: #가장 최근 정보 읽어오기
        conn = sqlite3.connect(db)
        with conn:
            cur = conn.cursor()
            cur.execute('select max(scrap_date) from airfare_scraped_data')
            scrap_date = cur.fetchone()[0]
        
    fares = []
    conn = sqlite3.connect(db)
    with conn:
        cur = conn.cursor()
        sitecode = ' or '.join(["scrap_site='"+c+"'" for c in sitecodes])
        sql = '''select airline,flt,dpt_date,dpt,arr,dpt_time,arr_time,fare,tax1,tax2,seat from airfare_scraped_data
                where scrap_date = '{scrap_date}' and {sitecode}'''
        print(sql.format(scrap_date=scrap_date,sitecode=sitecode))
        cur.execute(sql.format(scrap_date=scrap_date,sitecode=sitecode))
        rows = cur.fetchall()
        for row in rows:
            fares.append([row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10]])
    return fares
## 템플릿 이용 HTML 파일 작성
def make_grid_html(fare_data):
    with open('template/all_fare_grid.html',encoding='utf-8') as fp:
        template = fp.read()
        grid_data = make_grid_tr(fare_data)
    return template.replace('{grid_data}',grid_data).replace('\n','')

In [30]:
fare_list = read_all_fares('airfare_scraped_data.db',['IP'])

select airline,flt,dpt_date,dpt,arr,dpt_time,arr_time,fare,tax1,tax2,seat from airfare_scraped_data
                where scrap_date = '2017060815' and scrap_site='IP'


In [35]:
make_grid_html(fare_list)

'<!DOCTYPE html><!DOCTYPE html><html><head> <title>Fare Data Report From online scraping</title><meta charset="utf-8"><script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.1/jquery.min.js"></script><script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script><script src="https://ajax.googleapis.com/ajax/libs/angularjs/1.5.7/angular.min.js"></script><script src="https://ajax.googleapis.com/ajax/libs/angularjs/1.5.9/angular-route.js"></script><script src="http://d3js.org/d3.v4.min.js"></script><script type="text/javascript" charset="utf8" src="http://cdn.datatables.net/1.10.15/js/jquery.dataTables.js"></script><link rel="stylesheet" type="text/css" href="http://cdn.datatables.net/1.10.15/css/jquery.dataTables.css"><style type="text/css">\ttfoot input {\t\twidth: 100%;\t\tpadding: 3px;\t\tbox-sizing: border-box;\t}</style></head><body><h2>온라인 조회 내역 전체 리스트</h2><table id="table_id" class="display">    <thead><tr><th>airline</th><th>flt</th><th>date</th>